Skrypt do obliczania pokrycia masztów łączności systemu OPL i wizualizacji

In [153]:
# Załadowanie niezbędnych bibliotek Python3
import numpy as np
import matplotlib.pyplot as plt
import folium
%matplotlib inline

Parametry systemu łączności:

In [215]:
# Promień pokrycia masztu:
a = 5
# Dystans pomiędzy masztami:
b = 10
# Ilość masztów w systemie:
c = 5

In [216]:
# Obliczenia pokrycia przez system łączności

def single_node(r):
    return np.pi * r ** 2

def two_nodes(d, R, r):
    if d <= abs(R-r):
        # Jeden okrąg jest całkowicie zamknięty w drugim.
        return np.pi * min(R, r)**2
    if d >= r + R:
        # Oręgi w ogóle się nie pokrywają.
        nodes = np.pi * r ** 2 + np.pi * R ** 2
        return nodes

    r2, R2, d2 = r**2, R**2, d**2
    alpha = np.arccos((d2 + r2 - R2) / (2*d*r))
    beta = np.arccos((d2 + R2 - r2) / (2*d*R))
    A = (r2 * alpha + R2 * beta - 0.5 * (r2 * np.sin(2*alpha) + R2 * np.sin(2*beta)))
    nodes = np.pi * r ** 2 + np.pi * R ** 2 - A
    return nodes

def multi_nodes_in2line(d, R, r, n):
    if d <= abs(R-r):
        # Jeden okrąg jest całkowicie zamknięty w drugim.
        return np.pi * min(R, r)**2
    if d >= r + R:
        # Oręgi w ogóle się nie pokrywają.
        nodes = (np.pi * r ** 2) * n
        return nodes

    r2, R2, d2 = r**2, R**2, d**2
    alpha = np.arccos((d2 + r2 - R2) / (2*d*r))
    beta = np.arccos((d2 + R2 - r2) / (2*d*R))
    A = (r2 * alpha + R2 * beta - 0.5 * (r2 * np.sin(2*alpha) + R2 * np.sin(2*beta)))
    nodes = ((np.pi * r ** 2) * n) - (A * c)
    return nodes

single = round(single_node(a), 2)

if c<2:
    total = single
elif c==2:
    total = round(two_nodes(b, a, a), 2)
elif c>2:
    total = round(multi_nodes_in2line(b, a, a, c), 2)

Wizualizacja systemy łączności na mapie:

In [217]:
# Współrzędne na mapie dla M.St. Warszawy:
Warsaw = folium.Map(location=[52.2297700,21.0117800], zoom_start=10, tiles='Stamen Terrain')

# Dodawanie pokrycia masztów do mapy biorąc pod uwagę poniższe współczynniki konwersji:
# Latitude: 1 deg = 110.574 km
# Longitude: 1 deg = 111.320*cos(latitude) km = dla Warszawy 68.5 km, 
if c==1:
    folium.Circle(location=[52.2297700,21.0117800], radius=a*1000, color='blue', fill=True, fill_color='blue').add_to(Warsaw)
elif c==2:
    folium.Circle(location=[52.325899, 21.000582], radius=a*1000, color='blue', fill=True, fill_color='blue').add_to(Warsaw)
    latitude = 52.325899-(b/110.574)
    folium.Circle(location=[latitude,21.000582], radius=a*1000, color='blue', fill=True, fill_color='blue').add_to(Warsaw)
elif c>=3:
    d = [c // 2 + c % 2, c // 2]
    for i in range(d[0]):
        longitude = 21.1958900-(b/68.5*i)
        folium.Circle(location=[52.3395300,longitude], radius=a*1000, color='blue', fill=True, fill_color='blue').add_to(Warsaw)
    for j in range(d[1]):
        latitude = 52.3395300-(b/110.574)
        longitude = 21.1958900-(b/68.5*j)
        folium.Circle(location=[latitude,longitude], radius=a*1000, color='blue', fill=True, fill_color='blue').add_to(Warsaw) 

print("Powierzchnia M.St. Warszawy wynosi: 517.20 km2")
print(f"Pokrycie pojedyńczego masztu o zasięgu {a} km wynosi: {single} km2")
if c>1:
    print(f"Pokrycie {c} masztów o zasiegu {a} km każdy, oddalonych od siebie o {b} km wynosi: {total} km2")
Warsaw

Powierzchnia M.St. Warszawy wynosi: 517.20 km2
Pokrycie pojedyńczego masztu o zasięgu 5 km wynosi: 78.54 km2
Pokrycie 5 masztów o zasiegu 5 km każdy, oddalonych od siebie o 10 km wynosi: 392.7 km2


Dodatkowe obliczenia:

In [195]:
# Rozłożenie masztów:
d = [c // 2 + c % 2, c //2]
d

[3, 3]

In [209]:
# Przeliczenie 1 deg = 111.320*cos(latitude) km
np.cos(52 * np.pi / 180) * 111.320

68.53543543325227